In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [ ]:
df = pd.read_csv("train.csv")
df.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [ ]:
X = df["text"].copy()
authors = df["author"].copy()

# Label data
y = []
for author in authors:
    if author == "EAP":
        y.append([1, 0, 0])
    if author == "HPL":
        y.append([0, 1, 0])
    if author == "MWS":
        y.append([0, 0, 1])

y = np.array(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
encoder = tf.keras.layers.TextVectorization()
encoder.adapt(X_train)
vocab = encoder.get_vocabulary()

In [ ]:
def create_model(conv_filters, conv_size, lstm_units, dense_units):
  model = tf.keras.Sequential([
      encoder,
      tf.keras.layers.Embedding(
          input_dim=len(vocab),
          output_dim=64,
          # Use masking to handle the variable sequence lengths
          mask_zero=True),
      tf.keras.layers.Conv1D(filters=conv_filters,
                              kernel_size=conv_size,
                              padding="same",
                              activation="relu",
                              data_format="channels_last",
                              ),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences=False)),
      # tf.keras.layers.GlobalMaxPool1D(keepdims=False),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(3, activation="softmax")
  ])
  model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
            optimizer=tf.keras.optimizers.Adam(1e-3),
            metrics=['accuracy']
  )
  return model
  

In [ ]:
# Random search
# model = KerasClassifier(build_fn=create_model, batch_size=32, verbose=1)

# conv_filters = [int(i) for i in np.random.uniform(low=8, high=2048, size=3)]
# conv_size = [int(i) for i in np.random.uniform(low=2, high=16, size=3)]
# lstm_units = [int(i) for i in np.random.uniform(low=8, high=2048, size=3)]
# dense_units = [int(i) for i in np.random.uniform(low=8, high=2048, size=3)]

# params = dict(conv_filters=conv_filters, conv_size=conv_size, lstm_units=lstm_units, dense_units=dense_units)
# grid = GridSearchCV(estimator=model, scoring = 'neg_log_loss', param_grid=params, n_jobs=1, cv=3, verbose=3)

# grid_result = grid.fit(X_train, y_train)

# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
# Train model
# best_params = grid_result.best_params_
# model = create_model(best_params["conv_filters"], best_params["conv_size"], best_params["lstm_units"], best_params["dense_units"])
model = create_model(64, 5, 64, 64)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1494208   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          20544     
_________________________________________________________________
dropout (Dropout)            (None, None, 64)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0

In [ ]:
callbacks = [
          tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
          tf.keras.callbacks.ModelCheckpoint(
            filepath="BiLSTM_CNN_weights",
            save_weights_only=True,
            monitor='val_loss',
            mode='max',
            save_best_only=True)
]
with tf.device('/device:GPU:0'):
  history = model.fit(X_train, y_train, epochs=15,
                      validation_data=(X_test, y_test),
                      validation_steps=30,
                      callbacks=callbacks)

Epoch 1/15
490/490 [==============================] - 79s 150ms/step - loss: 0.6881 - accuracy: 0.6908 - val_loss: 0.4796 - val_accuracy: 0.8077
Epoch 2/15
490/490 [==============================] - 71s 145ms/step - loss: 0.2582 - accuracy: 0.9055 - val_loss: 0.4731 - val_accuracy: 0.8154
Epoch 3/15
490/490 [==============================] - 72s 147ms/step - loss: 0.1250 - accuracy: 0.9560 - val_loss: 0.6110 - val_accuracy: 0.8202
Epoch 4/15
490/490 [==============================] - 71s 145ms/step - loss: 0.0744 - accuracy: 0.9753 - val_loss: 0.7301 - val_accuracy: 0.8115
Epoch 5/15
490/490 [==============================] - 73s 148ms/step - loss: 0.0502 - accuracy: 0.9819 - val_loss: 0.8827 - val_accuracy: 0.8006
